# Đề tài: Web AI Quản Lý Tài Liệu Công Văn, Văn Bản Hành Chính

Hệ thống web ứng dụng AI để lưu trữ, quản lý, trích xuất thông tin và hỗ trợ ghi chú thông minh cho công văn, văn bản hành chính. Hệ thống hỗ trợ phân quyền truy cập, tích hợp AI tóm tắt, phân loại, và khai thác metadata tự động từ tài liệu số hóa.

## 1. Kiến trúc tổng thể AI_PROJECT

- **Backend:** Django (quản lý người dùng, phân quyền, API nội bộ), FastAPI (AI, tích hợp open-notebook, OCR, tóm tắt)
- **AI & NLP:** Tích hợp open-notebook, mô hình AI (OpenAI, Gemini, Ollama, v.v.)
- **Frontend:** Bootstrap, Chart.js, giao diện quản lý tài liệu, thống kê
- **Database:** Lưu trữ metadata, phân quyền, lịch sử truy cập
- **Tích hợp:** OCR, AI tóm tắt, trích xuất metadata, ghi chú AI

## 2. Cơ cấu tổ chức & phân quyền

- **Ban Giám Đốc:** Quản lý, điều hành tổng thể, duyệt/chỉnh sửa mọi tài liệu
- **Các phòng ban:** Văn phòng, Nhân sự, Tài chính kế toán, Nghiên cứu phát triển, Giám đốc dự án
- **Nhân viên:** Xem/tải tài liệu liên quan, đề xuất chỉnh sửa
- **Học viên:** Xem/tải tài liệu đào tạo đã đăng ký

**Phân quyền tài liệu:**  
- Ban Giám Đốc: Xem, tải, chỉnh sửa, duyệt tất cả  
- Trưởng phòng: Xem, tải, chỉnh sửa tài liệu phòng  
- Nhân viên: Xem, tải tài liệu liên quan, đề xuất chỉnh sửa  
- Giám đốc dự án: Xem, tải, chỉnh sửa tài liệu dự án  
- Học viên: Xem, tải tài liệu khóa học  

## 3. Phân loại tài liệu

- **Tài liệu chung:** Công văn đi đến, văn bản pháp quy, hợp đồng, báo giá
- **Tài liệu dự án:** Tài liệu dự án, báo cáo tiến độ, kết quả
- **Tài liệu đào tạo:** Tài liệu nội bộ, khóa học CNTT
- **Thư viện multimedia:** Hình ảnh, video, âm thanh

In [6]:
# Mô hình dữ liệu Django (rút gọn)
from django.db import models
from django.contrib.auth.models import User

class Department(models.Model):
    name = models.CharField(max_length=100)

class Position(models.Model):
    name = models.CharField(max_length=100)

class Profile(models.Model):
    user = models.OneToOneField(User, on_delete=models.CASCADE)
    department = models.ForeignKey(Department, on_delete=models.SET_NULL, null=True)
    position = models.ForeignKey(Position, on_delete=models.SET_NULL, null=True)

class DocumentCategory(models.Model):
    name = models.CharField(max_length=100)

class DocumentType(models.Model):
    name = models.CharField(max_length=100)
    category = models.ForeignKey(DocumentCategory, on_delete=models.CASCADE)

class Document(models.Model):
    title = models.CharField(max_length=200)
    document_type = models.ForeignKey(DocumentType, on_delete=models.SET_NULL, null=True)
    category = models.ForeignKey(DocumentCategory, on_delete=models.SET_NULL, null=True)
    file = models.FileField(upload_to='documents/')
    creator = models.ForeignKey(User, on_delete=models.SET_NULL, null=True)
    created_at = models.DateTimeField(auto_now_add=True)
    metadata = models.JSONField(default=dict)

ImproperlyConfigured: Requested setting INSTALLED_APPS, but settings are not configured. You must either define the environment variable DJANGO_SETTINGS_MODULE or call settings.configure() before accessing settings.

## 4. Luồng hoạt động chính

1. **Đăng nhập, phân quyền:** Người dùng đăng nhập, hệ thống xác định vai trò/phòng ban
2. **Tải lên tài liệu:** Hỗ trợ nhiều định dạng, tự động trích xuất metadata, OCR nếu là ảnh
3. **AI xử lý:** Gửi tài liệu đến FastAPI/open-notebook để tóm tắt, gắn thẻ, trích xuất thông tin
4. **Lưu trữ & phân quyền:** Lưu metadata, phân quyền truy cập/chỉnh sửa/tải về
5. **Tìm kiếm, thống kê:** Tìm kiếm toàn văn, lọc theo loại, phòng ban, dự án; thống kê số lượng, biểu đồ
6. **Ghi chú AI:** Sinh ghi chú, trả lời truy vấn, gợi ý câu hỏi liên quan tài liệu

In [3]:
# FastAPI endpoint tích hợp open-notebook (ví dụ)
from fastapi import FastAPI, UploadFile, File
from open_notebook import process_document

app = FastAPI()

@app.post("/ai/summary/")
async def ai_summary(file: UploadFile = File(...)):
    content = await file.read()
    result = process_document(content)
    return {
        "summary": result["summary"],
        "metadata": result["metadata"],
        "tags": result["tags"]
    }

ModuleNotFoundError: No module named 'open_notebook'

## 5. Giao diện quản lý tài liệu

- **Trang chủ:** Thống kê tổng quan, số lượng tài liệu, biểu đồ phân loại, hoạt động gần đây
- **Danh sách tài liệu:** Lọc, tìm kiếm, phân loại, tải về, xem chi tiết
- **Tải lên tài liệu:** Hỗ trợ nhiều định dạng, tự động trích xuất metadata
- **Chi tiết tài liệu:** Hiển thị nội dung, metadata, ghi chú AI, lịch sử chỉnh sửa
- **Phân quyền:** Quản lý quyền truy cập, chỉnh sửa, phê duyệt tài liệu

## 6. Bảo mật & phân quyền

- Phân quyền theo vai trò, phòng ban, dự án
- Tài liệu quan trọng cần phê duyệt trước khi chia sẻ
- Lưu lịch sử truy cập, chỉnh sửa
- Hỗ trợ xác thực hai lớp (2FA) nếu cần

## 7. Kết luận

Hệ thống Web AI quản lý công văn giúp tự động hóa lưu trữ, tìm kiếm, phân loại, và khai thác thông tin tài liệu hành chính, tăng hiệu quả quản lý và bảo mật thông tin cho tổ chức/doanh nghiệp.